In [1]:
import os
import gensim
import random

# Init corpus

In [56]:
all_doc = []

In [62]:
tagged_train_corpus = []
test_corpus = []

# Get Resumes

In [4]:
projectDir = 'D://pythonProject/Internship/Resume Screening/data/'
cleanResumeDir = projectDir+'new_preprocessed_txt_data/'

arr_dir = ['3D', 'Admin - CTV', 'AI', 'BLOCKCHAIN', 'C#', 'DEVOPS', 'HELPDESK', 'JAVA', 'KAGGLE', 'PHP', 'PYTHON', 'REACTJS',
          'RUBY', 'TESTER', 'UNITY', 'INTERN/OK', 'INTERN/PNV', 'INTERN/SUMMER INTERN 2021',
           'INTERN/SUMMER INTERN 2021/Intern Blockchain', 'NODEJS/I.2021', 'NODEJS/II.2021']

In [5]:
for dir in arr_dir:
    new_dir = cleanResumeDir+dir
    os.chdir(new_dir)
    for clean_resume in os.listdir(new_dir):
        if clean_resume == "Intern Blockchain":
            continue
        file = open(str(clean_resume), 'r')
        resume = file.read()
        all_doc.append(resume)

# Get JDs

In [27]:
projectDir = 'D://pythonProject/Internship/Resume Screening/data/'
clean_jd_dir = projectDir+'new_preprocessed_txt_JD/'

In [58]:
os.chdir(clean_jd_dir)
for clean_jd in os.listdir(clean_jd_dir):
    file = open(str(clean_jd), 'r')
    jd = file.read()
    all_doc.append(jd)

# Train-Test Split

In [63]:
len_doc = len(all_doc)
random.shuffle(all_doc)
i = 0
for i in range(len_doc):
    if i <= (len_doc*0.95):
        tagged_train_corpus.append(gensim.models.doc2vec.TaggedDocument(all_doc[i].split(), [i]))
    else:
        test_corpus.append(all_doc[i].split())
    i = i + 1

In [67]:
print(len(tagged_train_corpus), len(test_corpus))

35 1


In [70]:
tagged_train_corpus[1]

TaggedDocument(words=['html', 'css', 'javascript', 'mvc', 'cd'], tags=[1])

In [65]:
print(test_corpus[:1])

[[]]


# Init model

In [71]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=100)

In [72]:
model.build_vocab(tagged_train_corpus)

In [73]:
print(f"Word 'nodejs' appeared {model.wv.get_vecattr('nodejs', 'count')} times in the training corpus.")

Word 'nodejs' appeared 21 times in the training corpus.


# Train model

In [80]:
# run this if you have loaded the last model
model = model_resume

In [74]:
# jd: 1000 epochs
# resume: 1000 epochs
model.train(tagged_train_corpus, total_examples=model.corpus_count, epochs=1000)

In [75]:
ranks = []
second_ranks = []
for doc_id in range(len(tagged_train_corpus)):
    inferred_vector = model.infer_vector(tagged_train_corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    second_ranks.append(sims[1])

In [76]:
import collections

counter = collections.Counter(ranks)
print(counter)

Counter({0: 31, 1: 2, 34: 1, 33: 1})


In [77]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(tagged_train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(tagged_train_corpus[sims[index][0]].words)))

Document (34): «»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d100,n5,w5,mc2,s0.001,t3):

MOST (25, 0.23486721515655518): «node js php»

SECOND-MOST (16, 0.21202312409877777): «framework framework postgresql js docker»

MEDIAN (10, 0.06275653839111328): «php php php php laravel framework mvc php framework laravel cakephp mysql mongodb restful api nodejs reactjs vuejs cd docker»

LEAST (11, -0.18660326302051544): «reactjs vuejs vuejs html css javascript reactjs ux reactjs vuejs html css javascript es5 framework sass angular nodejs mongodb mysql ux restful api»



In [78]:
# Pick a random document from the corpus and infer a vector from the model
doc_id = random.randint(0, len(tagged_train_corpus) - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(tagged_train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(tagged_train_corpus[sim_id[0]].words)))

Train Document (2): «blockchain php nodejs reactjs php nodejs reactjs chance blockchain php nodejs reactjs blockchain deep php javascript blockchain php nodejs reactjs blockchain»

Similar Document (6, 0.8194918036460876): «php»



# TEST

In [79]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(tagged_train_corpus[sims[index][0]].words)))

Test Document (0): «»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d100,n5,w5,mc2,s0.001,t3):

MOST (11, 0.20748168230056763): «reactjs vuejs vuejs html css javascript reactjs ux reactjs vuejs html css javascript es5 framework sass angular nodejs mongodb mysql ux restful api»

MEDIAN (15, 0.010806656442582607): «nodejs php javascript entrepreneurial»

LEAST (30, -0.2499840408563614): «framework framework postgresql react docker»



# Save model

## Save the model for JD

In [80]:
modelDir = 'D://pythonProject/Internship/Resume Screening/Entity Extraction/Doc2Vec/model'
os.chdir(modelDir)
#SAVE THE MODEL
model_name = "model_jd_doc2vec_new"
model.save(model_name)

## Save the model for resume

In [19]:
modelDir = 'D://pythonProject/Internship/Resume Screening/Entity Extraction/Doc2Vec/model'
os.chdir(modelDir)
model_name = "model_resume_doc2vec_new"
model.save(model_name)

# Save dataset

In [21]:
import json

In [81]:
json_train = json.dumps(tagged_train_corpus)
json_test = json.dumps(test_corpus)

In [23]:
jsonFile = open("D://pythonProject/Internship/Resume Screening/Entity Extraction/Doc2Vec/data/new_resume_train_data.json", "w+")
jsonFile.write(json_train)
jsonFile.close()

In [24]:
jsonFile = open("D://pythonProject/Internship/Resume Screening/Entity Extraction/Doc2Vec/data/new_resume_test_data.json", "w+")
jsonFile.write(json_test)
jsonFile.close()

In [82]:
jsonFile = open("D://pythonProject/Internship/Resume Screening/Entity Extraction/Doc2Vec/data/new_jd_train_data.json", "w+")
jsonFile.write(json_train)
jsonFile.close()

In [83]:
jsonFile = open("D://pythonProject/Internship/Resume Screening/Entity Extraction/Doc2Vec/data/new_jd_test_data.json", "w+")
jsonFile.write(json_test)
jsonFile.close()

# Load dataset

In [101]:
import json
# run this to get the last dataset
file = open("D://pythonProject/Internship/Resume Screening/Entity Extraction/Doc2Vec/data/new_jd_train_data.json", 'r')
tagged_train_corpus = json.loads(file.read())

In [102]:
len(tagged_train_corpus)

35

In [103]:
for i in range(len(tagged_train_corpus)):
    tagged_train_corpus[i] = gensim.models.doc2vec.TaggedDocument(tagged_train_corpus[i][0], tagged_train_corpus[i][1])

In [104]:
tagged_train_corpus[0]

TaggedDocument(words=['postgresql', 'mysql', 'mongodb', 'cloudformation', 'docker', 'container'], tags=[0])

# Load model

In [3]:
import gensim
import os

projectDir = 'D://pythonProject/Internship/Resume Screening/'
modelDir = projectDir + 'Entity Extraction/Doc2Vec/model'
os.chdir(modelDir)

model_jd = gensim.models.Doc2Vec.load('model_jd_doc2vec_new')
model_resume = gensim.models.Doc2Vec.load('model_resume_doc2vec_new')

# Get vectors for new input data

In [60]:
from nltk.tokenize import word_tokenize
#to find the vector of a new input document
clean_jd = "D://pythonProject/Internship/Resume Screening/data/preprocessed_txt_JD/[BAP SOFTWARE] AI ENGINEER.txt"
file_jd = open(str(clean_jd), 'r')
test_data_jd = file_jd.read()
test_data_jd = test_data_jd.split()
print(test_data_jd)

['bap', 'software', 'co', 'ltd', '10f', '81', 'quang', 'trung', 'st', 'hai', 'chau', 'dist', 'da', 'nang', 'city', 'vietnam', 'website', 'https://bap-software.net/en/', 'ai', 'engineer', 'job', 'overview', 'bap', 'software', 'co', 'ltd', 'software', 'outsourcing', 'provider', 'one', 'subsidiaries', 'bap', 'group', 'always', 'aim', 'bring', 'innovation', 'products', 'customers', 'well', 'looking', 'talented', 'ai', 'engineer', 'da', 'nang', 'excited', 'part', 'winning', 'team', 'bap', 'great', 'place', 'grow', 'career', 'key', 'responsibilities', 'join', 'ai', 'team', 'ai', 'engineer', 'role', 'develop', 'projects', 'applying', 'ai', 'technology', 'design', 'implement', 'ai', 'technology', 'new', 'project', 'research', 'develop', 'ai', 'deep', 'learning', 'machine', 'learning', 'applications', 'assignments', 'key', 'skills', 'qualifications', 'models', '2', '+', 'years', 'experience', 'implementing', 'deep', 'learning', 'machine', 'learning', 'strong', 'ability', 'learn', 'technology', 

In [85]:
projectDir = 'D://pythonProject/Internship/Resume Screening/data/'
clean_jd_dir = projectDir+'new_preprocessed_txt_JD/'
all_jd = []
all_jd_name = []

In [86]:
os.chdir(clean_jd_dir)
for clean_jd in os.listdir(clean_jd_dir):
    file = open(str(clean_jd), 'r')
    all_jd_name.append(str(clean_jd))
    jd = file.read()
    all_jd.append(jd)

In [87]:
jd = []
for i in range(len(all_jd)):
    test_data_jd = all_jd[i].split()
    jd.append(model_jd.infer_vector(test_data_jd))
    jd[i] = jd[i].reshape(1, -1)
print(len(jd))

36


In [2]:
from nltk.tokenize import word_tokenize
#to find the vector of a new input document
clean_cv = "D://pythonProject/Internship/Resume Screening/data/new_preprocessed_txt_data/PHP/[Fresher] Bùi Hoàng Nhật - PHP.txt"
file_cv = open(str(clean_cv), 'r')
test_data_cv = file_cv.read()
test_data_cv = test_data_cv.split()
print(test_data_cv)

['php', 'php', 'php', 'php', 'php', 'phpmyadmin', 'mysql', 'php', 'mysql', 'mvc', 'laravel', 'framework', 'php', 'html', 'css', 'javascript', 'bootstrap', 'framework', 'laravel', 'mysql', 'php', 'mysql', 'laravel', 'framework', 'jquery', 'javascript', 'mysql', 'laravel', 'framework', 'html', 'css', 'bootstrap', 'php', 'mysql', 'php', 'combine', 'html', 'css', 'javascript']


In [5]:
resume = model_resume.infer_vector(test_data_cv)
resume = resume.reshape(1, -1)
print("Vector_cv_infer", resume)

Vector_cv_infer [[ 7.22881973e-01  8.37660015e-01 -5.23583516e-02 -1.28047180e+00
   4.35626179e-01  1.31364095e+00 -3.17696214e-01  6.43825591e-01
  -1.17543709e+00 -2.32136726e+00  1.35484233e-01 -7.66554415e-01
  -1.04958504e-01  2.31264487e-01  4.10977975e-02 -5.02311766e-01
   1.46337199e+00 -4.34674203e-01 -6.02834940e-01 -1.72340834e+00
  -1.12955141e+00 -3.36462975e-01  1.39101827e+00 -8.40078831e-01
  -7.57857203e-01 -8.04691434e-01  3.95499498e-01 -1.18264413e+00
  -5.79580143e-02  1.60710573e+00  3.15536976e-01 -2.04773974e+00
  -5.39182603e-01 -2.99074292e-01 -7.84700572e-01  2.09842682e+00
   7.29529858e-01  5.70261836e-01 -7.88897455e-01 -1.16113520e+00
   1.67042720e+00 -1.29253909e-01  7.89426029e-01 -3.24161440e-01
   9.05549765e-01 -8.67883027e-01  4.29419279e-01 -1.38015842e+00
  -8.86242390e-02  1.32013214e+00 -6.22016899e-02 -1.66017354e+00
   1.44575179e-01  5.17203748e-01 -3.32041264e+00 -2.57095963e-01
   9.78890300e-01 -1.34282720e+00 -7.98778832e-01 -1.2991653

# Cosine similarity

In [90]:
from sklearn.metrics.pairwise import cosine_similarity

# get the match percentage
matchPercentage = []
for i in range(len(jd)):
    matchPercentage.append(cosine_similarity(jd[i], resume)[0][0])
    matchPercentage[i] = round(matchPercentage[i], 2) # round to two decimal
    
index = [i for i, j in enumerate(matchPercentage)]
value = [j for i, j in enumerate(matchPercentage)]
top_match = sorted(zip(index, value), reverse=True, key=lambda x: x[1])[:]


for i in range(len(top_match)):
    print(all_jd_name[top_match[i][0]] + ": " + str(top_match[i][1]) + all_jd[top_match[i][0]])


[BAP SOFTWARE] NODEJS DEV.txt: 0.36 nodejs nodejs nodejs expressjs restful api mysql postgresql mongodb firebase reactjs angular vuejs docker
[BAP SOFTWARE] FRONTEND DEV.txt: 0.32 reactjs vuejs vuejs html css javascript reactjs ux reactjs vuejs html css javascript es5 framework sass angular nodejs mongodb mysql ux restful api
[BAP SOFTWARE] WEB_ APP DESIGNER.txt: 0.31 ux ux ux
[BAP SOFTWARE] C# DEV.txt: 0.27 winform framework mysql restful api docker cd
[BAP SOFTWARE] FULLSTACK DEV.txt: 0.27 nodejs php mongodb reactjs javascript reactjs nodejs php html css es5 expressjs laravel restful api mysql postgresql mongodb mvc firebase php angular vuejs cd docker
[BAP IT] BRSE.txt: 0.26 php
[BAP SOFTWARE] FULLSTACK JS DEV.txt: 0.25 javascript javascript javascript node.js react javascript ux gitlab nodejs react nodejs javascript reactjs html css javascript restful cd javascript
[BAP SOFTWARE] IT COMTOR.txt: 0.24 framework
[BAP SOFTWARE] QC_ TESTER.txt: 0.24 mysql api
[BAP SOFTWARE] SOFTWARE ARC